In [60]:
import os
import json
import numpy as np
from glob import glob

import torch
from torch.utils.data import Dataset, DataLoader

DATA_ROOT_PATH = "../eye_tracker_auto_labeller/data"

FACE_OVAL_LANDMARK_IDX_LIST = [
    162,  21,  54, 103,  67, 109,  10, 338, 297, 332, 284, 251, 389,
    356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148,
    176, 149, 150, 136, 172,  58, 132,  93, 234, 127, 162,
]
LEFT_EYE_LANDMARK_IDX_LIST = [
    263, 249, 390, 373, 374, 380, 381, 382, 362, 
    398, 384, 385, 386, 387, 388, 466, 263
]
LEFT_IRIS_LANDMARK_IDX_LIST = [475, 476, 477, 474, 475]
RIGHT_EYE_LANDMARK_IDX_LIST = [
     33,   7, 163, 144, 145, 153, 154, 155, 133,
    173, 157, 158, 159, 160, 161, 246, 33
]
RIGHT_IRIS_LANDMARK_IDX_LIST = [471, 472, 469, 470, 471]

class EyeTrackerDataset(Dataset) :
    def __init__(
        self,
        data_root_path = DATA_ROOT_PATH,
        face_oval_landmark_idx_list     = FACE_OVAL_LANDMARK_IDX_LIST,
        left_eye_landmark_idx_list      = LEFT_EYE_LANDMARK_IDX_LIST,
        left_iris_landmark_idx_list     = LEFT_IRIS_LANDMARK_IDX_LIST,
        right_eye_landmark_idx_list     = RIGHT_EYE_LANDMARK_IDX_LIST,
        right_iris_landmark_idx_list    = RIGHT_IRIS_LANDMARK_IDX_LIST
    ) :
        super(EyeTrackerDataset, self).__init__()
        self.DATA_ROOT_PATH = data_root_path
        self.face_oval_landmark_idx_list    = face_oval_landmark_idx_list
        self.left_eye_landmark_idx_list     = left_eye_landmark_idx_list
        self.left_iris_landmark_idx_list    = left_iris_landmark_idx_list
        self.right_eye_landmark_idx_list    = right_eye_landmark_idx_list
        self.right_iris_landmark_idx_list   = right_iris_landmark_idx_list
        self.file_path_list = sorted(glob(
            f"{eye_tracker_dataset.DATA_ROOT_PATH}/*/*.json"
        ))
    def __len__(self) :
        return len(self.file_path_list)
    
    def __getitem__(self, index):
        with open(self.file_path_list[index], "r") as fp :
            data = json.load(fp)
        mouse_position = torch.Tensor(data["mouse_position"])
        face_landmark_array = torch.Tensor(data["face_landmark_array"])[
            self.face_oval_landmark_idx_list +
            self.left_eye_landmark_idx_list +
            self.left_iris_landmark_idx_list +
            self.right_eye_landmark_idx_list +
            self.right_iris_landmark_idx_list
        ]
        return face_landmark_array, mouse_position
   


In [65]:
eye_tracker_dataset = EyeTrackerDataset(DATA_ROOT_PATH)

print(len(eye_tracker_dataset))
print(eye_tracker_dataset[1600][0].shape )

1612
torch.Size([81, 3])


In [53]:
import mediapipe as mp
mp_face_mesh = mp.solutions.face_mesh

def edge_list_2_path(edge_list) :
    tesel = edge_list
    change_occured = True
    while change_occured :
        change_occured = False
        for idx in range(len(tesel)) :
            target_edge = tesel[idx]
            inner_changed = False
            for e in tesel :
                if e != target_edge and len(e) < 3 :
                    edge = e
                    if target_edge[-1] == edge[0] :
                        target_edge.append(edge[-1])
                        tesel.remove(edge)
                        change_occured = True
                        inner_changed = True
                        break
                    if target_edge[0] == edge[-1] :
                        target_edge.insert(0, edge[0])
                        tesel.remove(edge)
                        change_occured = True
                        inner_changed  = True
                        break
            if inner_changed :
                break
    change_occured = True
    while change_occured :
        change_occured = False
        for idx in range(len(tesel)) :
            source_path = tesel[idx]
            inner_changed = False
            for target_path in tesel :
                if target_path == source_path :
                    continue
                if source_path[-1] == target_path[-1] :
                    target_path.reverse()
                    source_path += target_path[1:]
                    tesel.remove(target_path)
                    change_occured = True
                    inner_changed = True
                    break
                if source_path[0] == target_path[0] :
                    source_path.reverse()
                    target_path += source_path[1:]
                    tesel.remove(source_path)
                    change_occured = True
                    inner_changed = True
                    break
            if inner_changed :
                break
    return tesel


FACE_TESSELATION_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_TESSELATION
)).tolist())
FACE_OVAL_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_FACE_OVAL
)).tolist())
FACE_LIPS_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_LIPS
)).tolist())
FACE_LEFT_EYEBROW_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_LEFT_EYEBROW
)).tolist())
FACE_LEFT_EYE_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_LEFT_EYE
)).tolist())
FACE_LEFT_IRIS_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_LEFT_IRIS
)).tolist())
FACE_RIGHT_EYEBROW_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_RIGHT_EYEBROW
)).tolist())
FACE_RIGHT_EYE_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_RIGHT_EYE
)).tolist())
FACE_RIGHT_IRIS_PATH_LIST = edge_list_2_path(np.array(list(
    mp_face_mesh.FACEMESH_RIGHT_IRIS
)).tolist())



In [59]:
print(FACE_LEFT_EYE_PATH_LIST[0])
print()
print(FACE_LEFT_IRIS_PATH_LIST[0])
print()
print(FACE_RIGHT_EYE_PATH_LIST[0])
print()
print(FACE_RIGHT_IRIS_PATH_LIST[0])

[263, 249, 390, 373, 374, 380, 381, 382, 362, 398, 384, 385, 386, 387, 388, 466, 263]

[475, 476, 477, 474, 475]

[33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246, 33]

[471, 472, 469, 470, 471]
